In [121]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# transforemers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from kobert import get_pytorch_kobert_model
from kobert_transformers import get_kobert_model, get_distilkobert_model, get_tokenizer

# BERT 모델, Voca 불러오기
bertmodel, vocab = get_pytorch_kobert_model()
# bertmodel = get_kobert_model()


# GPU 사용
device = torch.device("cuda")

c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\mxnet\numpy\utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  bool = onp.bool


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_33008\3551097968.py", line 6, in <module>
    import gluonnlp as nlp
  File "c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\gluonnlp\__init__.py", line 23, in <module>
    from . import loss
  File "c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\gluonnlp\loss\__init__.py", line 23, in <module>
    from .activation_regularizer import *
  File "c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\gluonnlp\loss\activation_regularizer.py", line 25, in <module>
    from mxnet.gluon.loss import Loss
  File "c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\mxnet\__init__.py", line 33, in <module>
    from . import contr

In [81]:
import pandas as pd

# 데이터 로드 -> 리뷰 텍스트에 대한 긍정인지 부정인지 학습에 필요하므로 name컬럼 drop
Jungrang_reviews_sentiment = pd.read_excel('C:/Users/user/Desktop/saltlux_project/data/review_file_junrang.xlsx')
reviews_sentiment = Jungrang_reviews_sentiment[['content', 'sentiment']]
reviews_sentiment.head(2)

,content,sentiment
0,사가정 모인 이비인후과 항상 만원이다,0
1,좋아요,1


In [82]:
from sklearn.model_selection import train_test_split

X = reviews_sentiment[['content']]
y = reviews_sentiment['sentiment']

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [100]:
y.unique()

array([0, 1], dtype=int64)

In [101]:
# X와 y를 하나의 데이터프레임으로 합치기
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [43]:
# # Bert모델에 넣을 데이터셋 생성 -> transformer형식으로 데이터셋 변경 -> nlp 라이브러리 대신 transformers 라이브러리의 토크나이저를 사용
# class BERTDataset(Dataset):
#     def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
#         transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
#         self.sentences = [transform([i[sent_idx]]) for i in dataset]
#         self.labels = [np.int32(i[label_idx]) for i in dataset]

#     def __getitem__(self, i) :
#         return (self.sentences[i] + (self.labels[i], ))
    
#     def __len__(self):
#         return(len(self.labels))

In [84]:
# GPU 사용
device = torch.device("cuda")

In [110]:
# BERT 모델에 넣을 데이터셋 생성 -> transformer 형식으로 데이터셋 변경
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.sentences = [str(i[sent_idx]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) if not pd.isna(i[label_idx]) else 0 for i in dataset]

    def __getitem__(self, i):
        sentence = str(self.sentences[i])  # Ensure the sentence is a string
        encoding = self.tokenizer(sentence, padding=self.pad, truncation=True, max_length=self.max_len, return_tensors='np', return_token_type_ids=False)
        return {
            'input_ids': encoding['input_ids'][0],
            'attention_mask': encoding['attention_mask'][0],
            'labels': np.int32(self.labels[i])
        }

    def __len__(self):
        return len(self.labels)

In [118]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [105]:
# # 토큰화
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 직접 불러오기
# KoBERT 모델 및 토크나이저 불러오기
from transformers import BertModel
from kobert_transformers import get_tokenizer
model_name = 'skt/kobert-base-v1' #'monologg/kobert'
model = BertModel.from_pretrained(model_name)
# tokenizer = get_tokenizer()

# 토크나이저에 오류 발생 -> pip install protobuf 실행
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:
# X와 y를 하나의 데이터프레임으로 합치기
dataset_train = pd.concat([X_train, y_train], axis=1)
dataset_test = pd.concat([X_test, y_test], axis=1)

In [111]:
X_train = dataset_train[['content']]
y_train = dataset_train['sentiment']

tok = tokenizer

# 데이터셋 생성
data_train = BERTDataset(list(zip(X_train['content'], y_train)), 0, 1, tok, max_len, True, False)


In [112]:
X_test = dataset_test[['content']]
y_test = dataset_test['sentiment']

# 데이터셋 생성
data_test = BERTDataset(list(zip(X_test['content'], y_test)), 0, 1, tok, max_len, True, False)

In [113]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [116]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size = 768, num_classes=2, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()
    
    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [119]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer, schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
     {'params':[p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

c:\Users\user\Desktop\saltlux_project\transformer\torch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [122]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc/(batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc/(batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc/(batch_id+1)))